In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import pickle

In [2]:
## load the train model, scaler pickel, onehot
model = load_model('churn_model.h5')

#load the encoder amd scaler
with open('onehot_encoder_geography.pkl', 'rb') as file:
    onehot_encoder = pickle.load(file)
    
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder = pickle.load(file)
    
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [3]:
# example
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender':'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [4]:
geo_encoded = onehot_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder.get_feature_names_out(['Geography']))
geo_encoded_df

/Users/viruss/miniforge3/envs/TF/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
input_data = pd.concat([pd.DataFrame([input_data]).reset_index(drop=True), geo_encoded_df], axis=1)
input_data = input_data.drop(['Geography'], axis=1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [6]:
#encode gender
input_data['Gender'] = label_encoder.transform(input_data['Gender'])

In [7]:
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [8]:
input_scaled = scaler.transform(input_data)

In [9]:
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [10]:
#prediction
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


array([[0.04191631]], dtype=float32)

In [11]:
prediction_prob = prediction[0][0]
prediction_prob

np.float32(0.041916315)

In [12]:
if prediction_prob > 0.5:
    print('Customer is likely to churn')
else:
    print('Customer is likely not to churn')

Customer is likely not to churn
